In [17]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0,harsha2010:magellan:1.0.5-s_2.11,com.esri.geometry:esri-geometry-api:1.2.1,commons-io:commons-io:2.6,org.apache.spark:spark-streaming_2.11:2.2.0,org.apache.spark:spark-sql_2.11:2.2.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11",
        "spark.dynamicAllocation.enabled": false
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1619430760498_0006,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1619430760498_0006,spark,idle,Link,Link,✔


In [18]:
/**
 * @Description: a spatial join based on Filter-refine approach for NYC taxicab data
 * @author: Isam Al Jawarneh
 * @date: 02/02/2019
 *last update: 14/04/2021
 */

In [19]:
sc.version

res3: String = 2.2.0.2.6.3.84-1

In [20]:
import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.util.random.XORShiftRandom
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}

import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{
  DoubleType,
  StringType,
  StructField,
  StructType
}
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions.{collect_list, collect_set}
import org.apache.spark.sql.SQLContext
import org.apache.log4j.{Level, Logger}
import scala.collection.mutable
import scala.concurrent.duration.Duration
import java.io.{BufferedWriter, FileWriter}
import org.apache.commons.io.FileUtils
import java.io.File
import scala.collection.mutable.ListBuffer
import java.time.Instant
import org.apache.spark.util.CollectionAccumulator
import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.DataFrame

In [55]:
val containerStorageName = "meteomobilitysparkdenis-2021-04-24t15-54-44-656z"
val storageAccountName = "meteomobilityhdistorage"

storageAccountName: String = meteomobilityhdistorage

In [21]:
/////////////////////////////
/// Definition of schemas ///
/////////////////////////////

In [22]:
val parametersRegistrySchema = StructType(Array(
    StructField("IdParametro", StringType, false),
    StructField("PARAMETRO", StringType, false),
    StructField("UM", StringType, false),
    StructField("Tmed (min)", DoubleType, false),
    StructField("NOTE", StringType, false)))

parametersRegistrySchema: org.apache.spark.sql.types.StructType = StructType(StructField(IdParametro,StringType,false), StructField(PARAMETRO,StringType,false), StructField(UM,StringType,false), StructField(Tmed (min),DoubleType,false), StructField(NOTE,StringType,false))

In [23]:
val stationsRegistrySchema = StructType(Array(
    StructField("Stazione", StringType, false),
    StructField("Cod_staz", StringType, false),
    StructField("COMUNE", StringType, false),
    StructField("INDIRIZZO", StringType, false),
    StructField("PROVINCIA", StringType, false),
    StructField("Altezza", StringType, false),
    StructField("Id_Param", StringType, false),
    StructField("PARAMETRO", StringType, false),
    StructField("UM", StringType, false),
    StructField("Coord_X", DoubleType, false),
    StructField("Coord_Y", DoubleType, false),
    StructField("SR", StringType, false),
    StructField("LON_GEO", DoubleType, false),
    StructField("LAT_GEO", DoubleType, false),
    StructField("SR_GEO", StringType, false)))

stationsRegistrySchema: org.apache.spark.sql.types.StructType = StructType(StructField(Stazione,StringType,false), StructField(Cod_staz,StringType,false), StructField(COMUNE,StringType,false), StructField(INDIRIZZO,StringType,false), StructField(PROVINCIA,StringType,false), StructField(Altezza,StringType,false), StructField(Id_Param,StringType,false), StructField(PARAMETRO,StringType,false), StructField(UM,StringType,false), StructField(Coord_X,DoubleType,false), StructField(Coord_Y,DoubleType,false), StructField(SR,StringType,false), StructField(LON_GEO,DoubleType,false), StructField(LAT_GEO,DoubleType,false), StructField(SR_GEO,StringType,false))

In [24]:
val airDataSchema = StructType(Array(
    StructField("COD_STAZ", StringType, false),
    StructField("ID_PARAM", StringType, false),
    StructField("DATA_INIZIO", StringType, false),
    StructField("DATA_FINE", StringType, false),
    StructField("VALORE", DoubleType, false),
    StructField("UM", StringType, false)))

airDataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(COD_STAZ,StringType,false), StructField(ID_PARAM,StringType,false), StructField(DATA_INIZIO,StringType,false), StructField(DATA_FINE,StringType,false), StructField(VALORE,DoubleType,false), StructField(UM,StringType,false))

In [25]:
/////////////////////////////
///// Import Dataframes /////
/////////////////////////////

In [29]:
//"wasb[s]://<BlobStorageContainerName>@<StorageAccountName>.blob.core.windows.net/<path>"
val parametersRegistry = (spark.read.format("csv")
                          .option("header", "true")
                          .schema(parametersRegistrySchema)
                          .csv("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/registries/Parametri.csv")
                          .select($"IdParametro".as("Parameter_Id"),
                                  $"PARAMETRO".as("Parameter_Name"),
                                  $"UM".as("Unit_Of_Measurement"),
                                  $"Tmed (min)"))

parametersRegistry: org.apache.spark.sql.DataFrame = [Parameter_Id: string, Parameter_Name: string ... 2 more fields]

In [56]:
//"wasb[s]://<BlobStorageContainerName>@<StorageAccountName>.blob.core.windows.net/<path>"
val stationsRegistry = (spark.read.format("csv")
                        .option("header", "true")
                        .schema(stationsRegistrySchema)
                        .csv("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/registries/Stazioni Aria.csv")
                        .select($"Stazione".as("Station"), $"Cod_staz", $"COMUNE".as("City"), $"INDIRIZZO".as("Address"), $"PROVINCIA".as("Province"), $"LON_GEO", $"LAT_GEO")
                        .withColumn("Station_Code", regexp_replace($"Cod_staz", "\\.", ""))
                        .withColumn("Point", point($"LON_GEO",$"LAT_GEO"))
                        .drop("Cod_Staz", "LON_GEO", "LAT_GEO"))

stationsRegistry: org.apache.spark.sql.DataFrame = [Station: string, City: string ... 4 more fields]

In [57]:
val airData = (spark.read.format("csv")
               .option("header", "true")
               .schema(airDataSchema)
               .load("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/data/arpae_air_data")
               .select($"COD_STAZ".as("Station_Code"), $"ID_PARAM".as("Parameter_Id"), $"DATA_INIZIO", $"DATA_FINE", $"VALORE".as("Value"))
               .withColumn("Start_Date", to_timestamp($"DATA_INIZIO", "dd/MM/yyyy HH"))
               .withColumn("End_Date", to_timestamp($"DATA_FINE", "dd/MM/yyyy HH"))
               .drop("DATA_INIZIO", "DATA_FINE")
              )

airData: org.apache.spark.sql.DataFrame = [Station_Code: string, Parameter_Id: string ... 3 more fields]

In [58]:
airData.count()

res36: Long = 693891

In [59]:
//////////////////
/// Geohashing ///
//////////////////

In [60]:
// a user defined function to get geohash from long/lat point 
val geohashUDF = udf{(curve: Seq[ZOrderCurve]) => curve.map(_.toBase32())}

geohashUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(ArrayType(org.apache.spark.sql.types.ZOrderCurveUDT@158893c4,true))))

In [61]:
val precision = 30


precision: Int = 30

In [62]:
//getting plain data from CSV file (file with point Data Structure) and use UDF to get geohashes
val geohashedStations = (stationsRegistry
                         .withColumn("index", $"point" index  precision)
                         .withColumn("geohashArray1", geohashUDF($"index.curve")))
val explodedGeohashedStations = (geohashedStations
                                 .explode("geohashArray1", "geohash")
                                 { a: mutable.WrappedArray[String] => a })

explodedGeohashedStations: org.apache.spark.sql.DataFrame = [Station: string, City: string ... 7 more fields]

In [63]:
explodedGeohashedStations.select("Station", "Point", "index", "geohashArray1", "geohash").show(2,false)

+-------------------+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------+
|Station            |Point                                      |index                                                                                                                                                 |geohashArray1|geohash|
+-------------------+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------+
|GIARDINI MARGHERITA|Point(11.35406170088398, 44.48267113876953)|[[ZOrderCurve(11.348876953125, 44.4781494140625, 11.35986328125, 44.483642578125, 30, -4191424035449470976, 110001011101010100010010000011),Contains]]|[srbj43]     |srbj43 |
|GIARDINI MARGHERITA|Point(11.35406170088398

In [72]:
val rawBologna = (spark.read.format("magellan")
                  .option("type", "geojson")
                  .load("wasbs://" + containerStorageName + "@" + storageAccountName + ".blob.core.windows.net/data/geojson/Bologna_quartieri.geojson")
                  .select($"polygon", $"metadata"("NOMEQUART").as("Neighboorhood"))
                  )
val bologna = (rawBologna
               .withColumn("index", $"polygon" index  precision)
               .select($"polygon", $"index", $"Neighboorhood")
               .cache())
val zorderIndexedBologna = (bologna
                            .withColumn("index", explode($"index"))
                            .select("polygon", "index.curve", "index.relation","Neighboorhood")
                          )
val geohashedBologna = bologna.withColumn("geohashArray", geohashUDF($"index.curve"))
val explodedGeohashedBologna = geohashedBologna.explode("geohashArray", "geohash") { a: mutable.WrappedArray[String] => a }
explodedGeohashedBologna.count()

res51: Long = 426

In [77]:
//joining geohashed trips with exploded geohashed neighborhood using filter-and-refine approach (.where($"point" within $"polygon") is refine --> using the brute force method ray casting for edge cases or false positives)
val stationsInBologna = (explodedGeohashedBologna
                         .join(explodedGeohashedStations,
                               explodedGeohashedBologna("geohash") === explodedGeohashedStations("geohash"))
                         .where($"point" within $"polygon")
                        )
stationsInBologna.select("point").distinct.show(1000)

Exception in thread Thread-91:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 30



+--------------------+
|               point|
+--------------------+
|Point(11.35406170...|
|Point(11.340932, ...|
|Point(11.28508959...|
|Point(11.32752671...|
+--------------------+